## Classification with LeNet

In [1]:
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
from bigdl.dataset import mnist
from bigdl.util.common import init_engine, Sample

from bigdl.nn.layer import Linear, SpatialMaxPooling, \
    SpatialConvolution, ReLU, Sequential, Reshape, LogSoftMax
    
from bigdl.optim.optimizer import Optimizer, Adam, MaxEpoch, EveryEpoch, Top1Accuracy, \
    TrainSummary, ValidationSummary, SeveralIteration, SGD

from bigdl.nn.criterion import ClassNLLCriterion, CrossEntropyCriterion

In [2]:
sc

In [3]:
init_engine()

In [3]:
import pickle
import gzip

traffic_dir = 'traffic_signs_data'

with gzip.open(traffic_dir + '/train2.p.gz', mode='rb') as f:
    train = pickle.load(f)
with gzip.open(traffic_dir + '/test2.p.gz', mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['images'], train['labels']
X_test, y_test = test['images'], test['labels']

In [5]:
X_train.shape, y_train.shape

((39209, 32, 32, 3), (39209,))

In [6]:
CLASS_COUNT = len(np.unique(y_train))
assert len(np.unique(y_test)) == CLASS_COUNT

## Normalize data.
Use Min/Max normalization to improve convergence.

In [7]:
def normalize(image_data, labels, min_x=None, max_x=None):
    min_x = np.min(image_data) if min_x is None else min_x
    max_x = np.max(image_data) if max_x is None else max_x
    delta = max_x - min_x
    a, b = 0.1, 0.9
#     result = a + (image_data - min_x) * (b - a) / (max_x - min_x)
#     return result, min_x, max_x

    rdd_images = sc.parallelize(image_data)
    rdd_labels = sc.parallelize(labels)

    rdd_sample = rdd_images \
        .zip(rdd_labels) \
        .map(lambda (features, label): \
             Sample.from_ndarray((features - min_x) * (b - a) / delta, label + 1))
    return rdd_sample, min_x, max_x

In [8]:
X_train_norm, min_x, max_x = normalize(X_train, y_train)

**Important**: apply the same Min/Max values from the training set to the testing set:

In [9]:
X_test_norm, _, _ = normalize(X_test, y_test, min_x, max_x)

Inspect samples:

In [10]:
# samples = X_test_norm.collect()

In [11]:
# samples[0]

## LeNet-5

In [12]:
from bigdl.nn.initialization_method import Xavier, Zeros, RandomNormal

In [13]:
def LeNet(class_num):
    mu = 0.01
    sigma = 0.1
    w_init = RandomNormal(mu, sigma)
    
    model = Sequential()
    model.add(Reshape([3, 32, 32]))
    
    model.add(SpatialConvolution(3, 6, 5, 5).set_init_method(w_init, Zeros()).set_name('conv1'))
    model.add(ReLU())
    
    model.add(SpatialMaxPooling(2, 2, 2, 2).set_name('pool1'))
    
    model.add(SpatialConvolution(6, 16, 5, 5).set_init_method(w_init, Zeros()).set_name('conv2'))
    model.add(ReLU())
    
    model.add(SpatialMaxPooling(2, 2, 2, 2).set_name('pool2'))
    
    model.add(Reshape([400]))
    
    model.add(Linear(400, 120).set_init_method(w_init, Zeros()).set_name('fc1'))
    model.add(ReLU())

#     model.add(Linear(200, 120).set_name('fc1_5'))
#     model.add(ReLU())

    model.add(Linear(120, 84).set_init_method(w_init, Zeros()).set_name('fc2'))
    model.add(ReLU())

    model.add(Linear(84, class_num).set_init_method(w_init, Zeros()).set_name('logits'))
#     model.add(LogSoftMax())
    
    return model

In [14]:
lenet_model = LeNet(CLASS_COUNT)

creating: createRandomNormal
creating: createSequential
creating: createReshape
creating: createSpatialConvolution
creating: createZeros
creating: createReLU
creating: createSpatialMaxPooling
creating: createSpatialConvolution
creating: createZeros
creating: createReLU
creating: createSpatialMaxPooling
creating: createReshape
creating: createLinear
creating: createZeros
creating: createReLU
creating: createLinear
creating: createZeros
creating: createReLU
creating: createLinear
creating: createZeros


### Create training loop:

In [15]:
optimizer = Optimizer(model=lenet_model, training_rdd=X_train_norm,
    criterion=CrossEntropyCriterion(),
#                       criterion=ClassNLLCriterion(),
    optim_method=Adam(learningrate=1e-3, beta1=0.9, beta2=0.999, epsilon=1e-8),
#                       optim_method=SGD(nesterov=True, 
#                                        momentum=0.9, 
#                                        dampening=0.0,
#                                        learningrate=1e-4),
    end_trigger=MaxEpoch(30),
    batch_size=64)

creating: createCrossEntropyCriterion
creating: createAdam
creating: createMaxEpoch
creating: createOptimizer


In [16]:
# Set the validation logic
optimizer.set_validation(batch_size=64, 
                         val_rdd=X_test_norm,
                         trigger=EveryEpoch(),
                         val_method=[Top1Accuracy()])

creating: createEveryEpoch
creating: createTop1Accuracy


In [17]:
import shutil

app_name= 'lenet5-' # + datetime.now().strftime("%Y%m%d-%H%M%S")
# Remove old files:
base_path = os.path.join('/tmp/bigdl_summaries', app_name)
try:
    shutil.rmtree(base_path)
    shutil.rmtree('/private' + base_path)  # On Mac
except OSError:
    pass

train_summary = TrainSummary(log_dir='/tmp/bigdl_summaries', app_name=app_name)
train_summary.set_summary_trigger("Parameters", SeveralIteration(1))
val_summary = ValidationSummary(log_dir='/tmp/bigdl_summaries', app_name=app_name)
optimizer.set_train_summary(train_summary)
optimizer.set_val_summary(val_summary)

    
print("saving logs to ", app_name)

creating: createTrainSummary
creating: createSeveralIteration
creating: createValidationSummary
('saving logs to ', 'lenet5-')


#### Start training

In [18]:
%%time
trained_model = optimizer.optimize()
print("Done")

Done
CPU times: user 112 ms, sys: 56.9 ms, total: 169 ms
Wall time: 26min 47s
